In [1]:
import os
import sys


os.environ['MOMAPS_HOME'] = '/home/labs/hornsteinlab/Collaboration/MOmaps_Sagy/MOmaps'
os.environ['MOMAPS_DATA_HOME'] = '/home/labs/hornsteinlab/Collaboration/MOmaps/input'

sys.path.insert(1, os.getenv("MOMAPS_HOME"))
print(f"MOMAPS_HOME: {os.getenv('MOMAPS_HOME')}")


import numpy as np
import logging
import torch


import torch.backends.cudnn as cudnn

from src.common.lib.dataset import Dataset
from src.datasets.dataset_spd import DatasetSPD
from src.common.lib.utils import load_config_file
from src.common.lib.data_loader import get_dataloader
from sandbox.eval_new_arch.dino4cells.utils import utils
from sandbox.eval_new_arch.dino4cells.main_dino import DataAugmentationDINO

import matplotlib.pyplot as plt



%matplotlib inline

MOMAPS_HOME: /home/labs/hornsteinlab/Collaboration/MOmaps_Sagy/MOmaps
MOMAPS_HOME: /home/labs/hornsteinlab/Collaboration/MOmaps_Sagy/MOmaps
MOMAPS_HOME: /home/labs/hornsteinlab/Collaboration/MOmaps_Sagy/MOmaps
MOMAPS_HOME: /home/labs/hornsteinlab/Collaboration/MOmaps_Sagy/MOmaps


In [2]:
config_data_path = './src/datasets/configs/training_data_config/B78NoDSTrainDatasetAugInPlaceSubsetConfig'

config = {
        'seed': 1,
        'embedding': {
            'image_size': 224
        },
        'patch_size': 16,
        'num_channels': 2,
        # 'out_dim': 2,
        'use_bn_in_head': False,
        'norm_last_layer': True,
        
        'local_crops_number': 8,
        'warmup_teacher_temp': 0.02,
        'teacher_temp': 0.05,
        'warmup_teacher_temp_epochs': 1,
        'epochs': 100,
        'student_temp': 0.1,
        'center_momentum': 0.9,
        'momentum_teacher': 0.9,#6,#0.9,##0.996,
        
        'lr':0.0500,#0.05,# 1e-1,# 0.0005,# 1e-4,##,
        'min_lr': 0.005000, #5e-2,#1e-5,#1e-6,
        'warmup_epochs': 2,##0,#10,#1,
        
        'weight_decay': 0.01,#1e-7,#0.04,#0.04,
        'weight_decay_end': 0.1,#1e-6,#0.4,#0.4,
    
        'freeze_last_layer': 1,#1,
        
        
        'batch_size_per_gpu': 10,
        'num_workers': 1,
        
        'accumulation_steps': 5,
        
        'logs_dir':"/home/labs/hornsteinlab/Collaboration/MOmaps_Sagy/MOmaps/sandbox/eval_new_arch/dino4cells/logs",
        'tensorboard_root_folder': "/home/labs/hornsteinlab/Collaboration/MOmaps_Sagy/MOmaps/sandbox/eval_new_arch/dino4cells/tensorboard"
    }

In [3]:
utils.fix_random_seeds(config.seed)

cudnn.benchmark = False

# ============ preparing data ... ============
transform = DataAugmentationDINO(config=config)


config_data = load_config_file(config_data_path)

dataset_train = DatasetSPD(config_data, transform)
config.out_dim = len(np.unique(dataset_train.label))
train_indexes, val_indexes, test_indexes = None, None, None
# dataset_val, dataset_test = None, None


# if is_one_config_supplied:
# dataset_val, dataset_test = deepcopy(dataset_train), deepcopy(dataset_train) # the deepcopy is important. do not change. 
# dataset_test.flip, dataset_test.rot = False, False
if config_data.SPLIT_DATA:
    logging.info("Split data...")
    train_indexes, val_indexes, test_indexes = dataset_train.split()
# else:
#     dataset_val, dataset_test = DatasetSPD(config_val), DatasetSPD(config_test)


################# TAKING ONLY PART OF THE DATA ################
quick_train_indices = train_indexes[:]
quick_val_indices = val_indexes[:]
# logging.info(f"[WARNING!] TAKING ONLY PART OF THE DATA!! train: {len(quick_train_indices)} val: {len(quick_val_indices)}")
############################################################



# quick_val_indices = val_indexes[:100]
dataset_train_subset = Dataset.get_subset(dataset_train, quick_train_indices)
dataset_val_subset = Dataset.get_subset(dataset_train, quick_val_indices)
dataset = dataset_train_subset
dataset_val = dataset_val_subset

data_loader = get_dataloader(dataset, config.batch_size_per_gpu, num_workers=config.num_workers)

AttributeError: 'dict' object has no attribute 'seed'